In [1]:
import os 
import re
import sys
sys.path.append('..')
from classes.Parser import Parser
import pandas as pd
from classes.Refactoring_machine import Refactoring_machine

In [2]:
rm = Refactoring_machine()
rm.extract_function('../test_data/case1.py','../test_results/test4.txt',20,23)


[24, 25, 26, 27, 28]


(True, 'Succes')

In [2]:
def test3():
    rm = Refactoring_machine()
    return rm.change_name('../test_data/case1.py','../test_results/test3.txt','main','func1','new_func')
test3()

(True, 'Succes')

In [8]:
rm = Refactoring_machine()
rm.change_name('../test_data/case1.py','../test_results/test7.txt','main.func1','func2','new_func')

(True, 'Succes')

In [3]:
def test4():
    rm = Refactoring_machine()
    return rm.extract_function('../test_data/case1.py','../test_results/test4.txt',9,11)
test4()

[11, 12]
14
b
11
0


(True, 'Succes')

In [2]:
def test2():
    rm = Refactoring_machine()
    return rm.change_name('../test_data/case1.py','../test_results/test2.txt','main','func2','new_func')
test2()

(True, 'Succes')

In [10]:
str = 'main.a.lol'
num = str.rfind('.')
str[num+1:]

'lol'

In [2]:
rm = Refactoring_machine()

In [2]:
rm.change_name('../test_data/case1.py','./wreszcie.txt','main','func1','hah')

NameError: name 'rm' is not defined

['class B():\n',
 '    def __init__(self) -> None:\n',
 '        pass\n',
 '\n',
 'class A(B):\n',
 '    def __init__(self) -> None:\n',
 '        pass\n',
 '    \n',
 '    def method(a,b):\n',
 '        c = a + b\n',
 '        return c\n',
 '    \n',
 '    def method2(self,g,cb):\n',
 '        o = self.method(g,cb)\n',
 '        return o\n',
 '\n',
 'o = A()\n',
 '\n',
 '\n',
 'def func1():\n',
 '    a = 3\n',
 '    b = 4\n',
 '    c =4\n',
 '    g = a+ 3\n',
 '    d = b +c + a+ 5\n',
 '    f =11\n',
 '    o = g+d\n',
 '    \n',
 '\n',
 't = a *3\n',
 'gaba = a+4\n',
 'for i in range(1):\n',
 '    for j in range( a + b ):\n',
 '        print(gaba)']

In [3]:
with open('../test_data/case1.py') as f:
    lines = f.readlines()
lines

pars = Parser()
data =  pd.DataFrame(pars.get_data(lines)).T
data = pars.parse_functions(data)
data

,original_text,level_num,kind,line_text,name,arguments,result,functions
0,\n,0,None,,None,[],NaN,main
1,"def func1(a,b):\n",0,def,"def func1(a,b):",func1,None,NaN,main
2,c = a + b\n,1,None,c = a + b,None,"[a, b]",c,main.func1
3,a = 3\n,1,None,a = 3,None,[3],a,main.func1
4,def func2(a):\n,1,def,def func2(a):,func2,None,NaN,main.func1
5,a = a*4\n,2,None,a = a*4,None,"[a, 4]",a,main.func1.func2
6,return a\n,2,return,return a,None,[a],NaN,main.func1.func2
7,a = 1\n,1,None,a = 1,None,[1],a,main.func1
8,d = 0\n,1,None,d = 0,None,[0],d,main.func1
9,b = 3 + d + a\n,1,None,b = 3 + d + a,None,"[3, d, a]",b,main.func1


In [26]:
def take_application_index(function,data):
    original_id  = function.name
    fname = function['name']
    index_of_occur = []
    index_to_exclude = []
    for i,row in data.iterrows():
            if row['kind'] == 'def' and row['name'] == fname and i > original_id:
                omitted  = take_application_index(data.iloc[i],data)
                index_to_exclude =  index_to_exclude + omitted
                index_to_exclude.append(i)
            if function.functions in row.functions and row['name'] == fname and i >= original_id:
                index_of_occur.append(i)
    return [x for x in index_of_occur if  x  not in index_to_exclude]
func = data.loc[(data['name'] == 'func1') & (data.kind =='def')].iloc[0]
data.iloc[take_application_index(func,data)]

,original_text,level_num,kind,line_text,name,arguments,result,functions
20,def func1():\n,1,def,def func1():,func1,None,NaN,main.func2


In [8]:
data.loc[(data['name'] == 'func1') & (data.kind =='def')].iloc[0]

original_text    def func1():\n
level_num                     0
kind                        def
line_text          def func1():
name                      func1
arguments                  None
result                      NaN
functions                  main
Name: 19, dtype: object

In [5]:
def parse_functions(data):
    last_f_name = {}
    last = ['main']
    for i,row in data.iterrows():
        #print(row['name'],row['level_num'],last)
        
        if row['level_num'] == 0:
            last_f_name[i] = 'main'
        else:
            #print('tutaj')
            temp = ''
            for str in last:
                temp = temp + '.' + str 
            last_f_name[i] = temp[1:]
            
        if row.kind == 'def' or row.kind == 'class':
            last.append(row['name'])
            #print('tutaj',row['name'])
        
            
        if row['kind'] == 'return':
            last.pop()
            
        def change(str):
            if str is not None:
                return str.replace(' ','')
            else:
                return None
        data['functions'] = pd.Series(last_f_name)
        data['functions'] = data['functions'].apply( lambda x: change(x))
        data['name'] = data['name'].apply( lambda x: change(x))
        return data

In [9]:
def rename_function(data,function_path,fname,name):
    index_of_occur = []
    for i,row in data.iterrows():
        if function_path in row.functions and row['name'] == fname:
            index_of_occur.append(i)
    new_lines_of_text = {}
    for i in data.index.to_list():
        if i in index_of_occur:
            new_lines_of_text[i] = data.iloc[i].line_text.replace(fname,name)
        else:
            new_lines_of_text[i] = data.iloc[i].line_text
    data.line_text = pd.Series(new_lines_of_text)
    return data

rename_function(data,'main','func1','hah')
    

,original_text,level_num,kind,line_text,name,arguments,result,functions
0,\n,0,None,,None,[],NaN,main
1,"def func1(a,b):\n",0,def,"def hah(a,b):",func1,None,NaN,main
2,c = a + b\n,1,None,c = a + b,None,"[a, b]",c,main.func1
3,a = 3\n,1,None,a = 3,None,[3],a,main.func1
4,def func2(a):\n,1,def,def func2(a):,func2,None,NaN,main.func1
5,a = a*4\n,2,None,a = a*4,None,"[a, 4]",a,main.func1.func2
6,return a\n,2,return,return a,None,[a],NaN,main.func1.func2
7,a = 1\n,1,None,a = 1,None,[1],a,main.func1
8,d = 0\n,1,None,d = 0,None,[0],d,main.func1
9,b = 3 + d + a\n,1,None,b = 3 + d + a,None,"[3, d, a]",b,main.func1


In [10]:
def get_paths(data):
    last_function = 'main'
    new_funtions = {}
    last_level = 0
    last_kind = ''
    for i,row in data.iterrows():
        func = row.functions
        lev = row.level_num
        for func in func
        
        if lev > last_level :
        #     last_function = f'{last_function}.{func}'
        #     new_funtions[i] = last_function
        # if lev > last_level and last_kind == 'for':
        #     new_funtions[i] = last_function
        # elif (lev < last_level):
        #     splits = last_function.split('.')
        #     last_function = ''
        #     for split in splits[: lev]:
        #         last_function = last_function +'.'+split
        #     last_function = last_function[1:]
        #     new_funtions[i] = last_function
        # elif lev == last_level:
        #     new_funtions[i] = last_function
        # elif lev == 0:
        #     new_funtions[i],last_function = 'main','main'
        # last_level = lev
        # last_kind = row.kind
    return pd.Series(new_funtions)
get_paths(data)
        

SyntaxError: invalid syntax (Temp/ipykernel_22360/1133219579.py, line 9)

In [ ]:
for func

In [ ]:
def take_arguments_from_box(data,box):
    arg = []
    for i in range(box[0],box[1]):
        n_args = [ find_argument(data,x,i,data.iloc[i].functions) for x in data.iloc[i].arguments if not x.isnumeric() and "'" not in x and '"' not in x]
        arg = arg + n_args
    return arg

def take_argument_from_box(data,box):
    arg = []
    for i in range(box[0],box[1]):
        n_args = [ find_argument(data,x,i,data.iloc[i].functions) for x in data.iloc[i].arguments if not x.isnumeric() and "'" not in x and '"' not in x]
        arg = arg + n_args
    return arg
    
    
def find_argument(data,argument,line_num, function):
    _ = data.loc[(data.result == argument )& (data.functions == function)]
    potential_lines = [x for x in _.index.to_list() if x < line_num]
    potential_lines.sort()
    return potential_lines[-1]

take_arguments_from_box(n_data,(8,11))

[7, 6]

In [ ]:
def take_prerequesits(data,line_num):
    
    def _help_func(data,line_num):
        arguments = data.iloc[line_num].arguments
        prereq = data.loc[data.result.isin(arguments)].index.to_list()
        prereq = [x for x in prereq if x < line_num]
        if len(prereq) == 0:
            return []
        grand = []
        for i in prereq:
                _ = _help_func(data,i)
                if len(_) != 0:
                    grand.append(_[-1])
                    
        rv =  grand + prereq
        return rv
    prereq = _help_func(data,line_num)
    temp_lines = data.iloc[prereq]
    rv = []
    for res in temp_lines.result.unique():
        result_type_list = temp_lines.loc[temp_lines.result == res].index.to_list()
        result_type_list.sort()
        rv.append(result_type_list[-1])
    
    return rv
    
data.iloc[pd.unique(take_prerequesits(data,14))]

,original_text,level_num,kind,line_text,name,arguments,result,functions
7,d = 0\n,0,None,d = 0,None,[0],d,main
6,a = 1\n,0,None,a = 1,None,[1],a,main
8,b = 3 + d + a\n,0,None,b = 3 + d + a,None,"[3, d, a]",b,main


In [ ]:
l = [1]
l[-1]

1

In [ ]:
def take_prerequesits_from_box(data,box):
    list_of_lines = data.loc[data.functions == data.iloc[box[1]].functions].index.to_list()
    list_of_lines = [x for x in list_of_lines if x > box[1]]
    prereq = []
    for i in list_of_lines:
        f = data.iloc[i].functions
        for arg in data.iloc[i].arguments:
            if not arg.isnumeric() and "'" not in arg and '"' not in arg:
                prereq = prereq + [find_argument(data,arg,i,f)]
    
    prereq = pd.unique(prereq).tolist()
    prereq = [x for x in prereq if x in range(box[0],box[1])]
    return prereq

take_prerequesits_from_box(n_data,(7,9))

[8]

In [ ]:
def make_function(data,start_line,end_line,name):
    if data.iloc[start_line].level_num != data.iloc[end_line + 1].level_num or data.iloc[start_line].functions != data.iloc[end_line + 1].functions:
        return False
    returns = take_prerequesits_from_box(data,(start_line,end_line))
    arguments = take_arguments_from_box(data,(start_line,end_line))
    arguments = data.iloc[arguments].result.to_list()
    returns = data.iloc[returns].result.to_list()
    if len(returns) == 0:
        pass
    replace = "'"
    
    start_index = data.loc[data.functions == data.iloc[start_line].functions].index.to_list()[0]
    def_string = f"def {name}({str(arguments)[1:-1].replace(replace,'')}):"
    start_level = data.iloc[start_line].level_num
    fun ={start_index: [def_string,start_level]} 
    
    start_index_backup = start_index 
    
    
    for i in range(start_line,end_line):
        start_index +=1
        fun[start_index] = [data.iloc[i].line_text,start_level +1]
    
    if len(returns) != 0:
        start_index +=1
        fun[start_index] = [f"return {str(returns)[1:-1].replace(replace,'')}",start_level+1]
        func_str = f"{str(returns)[1:-1].replace(replace,'')} = {name}({str(arguments)[1:-1].replace(replace,'')})"
    else:
        func_str = f"{name}({str(arguments)[1:-1].replace(replace,'')})"
        
    envoction = pd.DataFrame([{'line_text':func_str,'level_num':start_level}])
    
       
    
    
    my_function = pd.DataFrame(fun).T
    my_function.columns=['line_text','level_num']
    
    po = data.loc[data.index >  end_line][['line_text','level_num']]
    przed = data.loc[data.index <  start_line][['line_text','level_num']]
        
   
    #print(func_str)
    changed = pd.concat([przed,envoction,po]).reset_index(drop=True)
    przed = changed.iloc[changed.index < start_index_backup]
    po = changed.iloc[changed.index > start_index_backup]
    return pd.concat([przed,my_function,po]).reset_index(drop=True)
    


    #print(f"return {str(returns)[1:-1].replace(replace,'')}")
make_function(data,7,9,'lala')

,line_text,level_num
0,"def lala(d, a):",0
1,d = 0,1
2,b = 3 + d + a,1
3,return b,1
4,"def func1(a,b):",0
5,c = a + b,1
6,return c,1
7,,0
8,a = 3,0
9,a = 1,0


In [ ]:
lista = [1,1,12,3]
lista.pop()
lista

[1, 1, 12]

In [ ]:
def get_functions

In [ ]:
_ = data.iloc[:9]
data.iloc[:3]

,original_text,level_num,kind,line_text,name,arguments,result,functions
0,\n,0,None,,None,[],NaN,main
1,"def func1(a,b):\n",0,def,"def func1(a,b):",func1,None,NaN,main
2,c = a + b \n,1,None,c = a + b,None,"[a, b]",c,func1


In [ ]:
def look_for_assign(line:str):
    if '=' in  line:
        splits = line.split('=')
        splits[0] = splits[0].replace(' ','')
        return True,splits
    else:
        return False, 0

In [ ]:
def get_level(line):
    return line.count('    ')

In [ ]:
def split_by_operators(line):
    operators = ['-','+','*','/']
    for operator in operators:
        line = line.replace(operator,'___TEMPORARY_SPLIT___')
    line = line.replace(' ','')
    return line.split('___TEMPORARY_SPLIT___')

In [ ]:
def get_function(line):
    if line[0:5] == 'class':
        
        if '(' in line and ')' in line:
            parent = line[line.find("(")+1:line.find(")")]
            class_name = line[5:line.find("(")]
        else:
            parent = None
            class_name = line[5:]
        return 'class', class_name,parent
    elif line[0:3] == 'for':
        iterators = line[line.find("for")+3:line.find("in")]
        iterators = iterators.replace(" ",'')
        iterators = iterators.split(',')
        arguments = None
        if '(' and ')' in line:
            arguments = line[line.find("(")+1:line.find(")")]
            arguments = split_by_operators(arguments)
            if arguments[0] == '':
                arguments = None
        return 'for', iterators,arguments
    elif line[0:3] == 'def':
        name = line[line.find("def")+3:line.find("(")]
        return 'def',name,None
    elif '('in line and ')' in line:
        name =line[:line.find("(")]
        arguments = line[line.find("(")+1:line.find(")")]
        arguments = split_by_operators(arguments)
        if arguments[0] == '':
            arguments = None
        return 'func',name,arguments
    return None,None,None
    
get_function('a.print()')


('func', 'a.print', None)

In [ ]:
def get_data(lines):
    data = {}
    for i,line in enumerate(lines):
        row = {}
        row['original_text'] = line
        row['level_num'] = get_level(line)
        row['kind'] = None
        line = line.replace('    ','')
        line = line.replace('\n','')
        row['line_text'] = line
        assign_flag,ass_res = look_for_assign(line)
        if assign_flag:
            row["resoult"] = ass_res[0]
            line = ass_res[1]
        kind,name,arguments = get_function(line)
        row['kind'] = kind
        row['name'] = name
        if kind is None:
            arguments = split_by_operators(line)
        row['arguments'] = arguments
        data[i] = row
        #print (split_by_operators(line), data['original_text'])
        #split_by_operators(line)
    return data
        
get_data(lines)

{0: {'original_text': '\n',
  'level_num': 0,
  'kind': None,
  'line_text': '',
  'name': None,
  'arguments': ['']},
 1: {'original_text': 'a = 1\n',
  'level_num': 0,
  'kind': None,
  'line_text': 'a = 1',
  'resoult': 'a',
  'name': None,
  'arguments': ['1']},
 2: {'original_text': 'b = 4\n',
  'level_num': 0,
  'kind': None,
  'line_text': 'b = 4',
  'resoult': 'b',
  'name': None,
  'arguments': ['4']},
 3: {'original_text': 'c = 10\n',
  'level_num': 0,
  'kind': None,
  'line_text': 'c = 10',
  'resoult': 'c',
  'name': None,
  'arguments': ['10']},
 4: {'original_text': 'd = 14\n',
  'level_num': 0,
  'kind': None,
  'line_text': 'd = 14',
  'resoult': 'd',
  'name': None,
  'arguments': ['14']},
 5: {'original_text': 't = a *3\n',
  'level_num': 0,
  'kind': None,
  'line_text': 't = a *3',
  'resoult': 't',
  'name': None,
  'arguments': ['a', '3']},
 6: {'original_text': 'gaba = a+4\n',
  'level_num': 0,
  'kind': None,
  'line_text': 'gaba = a+4',
  'resoult': 'gaba',
  